# Using Python with Excel (and csv) files

Our objective is to present some practical ways Python can be used to automate working with Excel files. Some tools for doing this include:

* pandas - [read_excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html), [ExcelWriter](https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html), [to_excel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html)
* [openpyxl](https://openpyxl.readthedocs.io/en/stable/) - can read, write, and modify Excel files
* [XlsxWriter](https://xlsxwriter.readthedocs.io/) - powerful but can't edit existing Excel files https://xlsxwriter.readthedocs.io/working_with_tables.html
* [xlwings](https://www.xlwings.org/) - provides "glue" between Python and Excel

**Note:** It appears that [xlrd](https://pypi.org/project/xlrd/) and [xlwt](https://pypi.org/project/xlwt/) are not longer being maintained (they're only for `xls` files) though they are still in Anaconda. Use openpyxl instead.

Along the way, we are going to learn about and use the Python built in library, [pathlib](https://docs.python.org/3/library/pathlib.html), for working with file and folder paths.

## Good resources (many more scattered throughout)

* https://pbpython.com/
* https://www.datacamp.com/community/tutorials/python-excel-tutorial
* Excel Hell presentation: https://github.com/chris1610/pbpython/blob/master/presentations/Escaping-Excel-Hell-with-Python-and-Pandas.pdf


In [ ]:
import pandas as pd

## Prelude - Review of reading json, csv and Excel files with Python
Back in the pcda class, I included some optional material on reading json, csv, and Excel files using Python. In those short lessons, I also included some good info on using PyCharm effectively and basic package management with conda and pip. Before launching into new material, I encourage you to take a quick look at these for a refresher. For your convenience, I've included the notebooks and Python scripts in this module's Downloads file. You can find the screencasts at:

* http://www.sba.oakland.edu/faculty/isken/courses/mis5470_w21/python_intro_2.html#optional-advanced-activities



## Combining and splitting of data in csv and Excel files
We will explore a few examples motivated by common scenarios such as:

* You have a whole folder full of csv (or Excel) files with the same file structure and you need to combine them into a single file. You might also need to make some changes to the consolidated file.
* You have an Excel file with multiple sheets of similarly structured data and you want to consolidate them into a single sheet.
* You have an Excel file with data in "wide" format and you need to convert it to long format, and then perhaps export out individual files (one per the key column(s) in the long formatted data).
* You have an Excel file acting as a simple flatfile database. Periodically, you get new Excel files that need to get appended to the "database" file.

## Example 1 - Concatenating csv files

As part of a [research project on hospital capacity planning](http://hselab.org/comparing-predictive-models-for-obstetrical-unit-occupancy-using-caret-part-1.html), I ended up
running many computer simulations. For example, there were 150 different
scenarios, each requiring 25 runs (replications) of the simulation model. Each
run generated a log file that was subsequently processed by a Python script. Much
of this post-processing took place in parallel threads with a subset of the
scenarios allocated to each thread. After all that, I ended up with a
subdirectory containing a number of csv files with identical structure that
needed to be combined into one big csv file. Here's a little bit of one of
the files.

```
  scenario,rep,timestamp,num_days,num_visits_obs,num_visits_ldr,num_visits_pp, ...
  30,1,2016-03-04 12:44:29.610563,105553,42150.0,42154.0,42159.0,8409.0, ...
  30,10,2016-03-04 12:48:00.519651,105553,42288.0,42296.0,42305.0,8500.0, ...
  30,11,2016-03-04 12:51:30.571655,105553,42210.0,42209.0,42227.0,8387.0, ...
  30,12,2016-03-04 12:55:08.550450,105553,42420.0,42433.0,42457.0,8341.0, ...
  30,13,2016-03-04 12:58:48.635709,105553,42277.0,42289.0,42301.0,8429.0, ...
  30,14,2016-03-04 13:02:21.325709,105553,42127.0,42135.0,42150.0,8581.0, ...
  ... many more rows and columns
```

I also wanted to make sure that the final combined file was sorted in
ascending order by scenario by replication. Even though I only had a handful
of files, manually doing the concatenation using Excel and copying and pasting
got old quickly. Also, I often face this same problem but have hundreds of csv
files to concatenate. Seems like a good excuse to put together a Python script
to address this problem.

Here is a directory listing of the files to be concatenated.


```
 Directory of C:\Users\isken\Documents\teaching\aap_s21\excel_with_python\data\sim_output

03/31/2021  06:56 AM    <DIR>          .
03/31/2021  06:56 AM    <DIR>          ..
02/26/2016  01:15 PM           260,332 results_Exp9_Tandem05_nodischadj_0.csv
02/27/2016  06:49 AM         1,466,932 results_Exp9_Tandem05_nodischadj_100.csv
02/26/2016  07:18 PM           585,564 results_Exp9_Tandem05_nodischadj_1_2.csv
02/26/2016  11:05 PM           876,058 results_Exp9_Tandem05_nodischadj_3_5.csv
02/27/2016  02:16 AM         1,170,356 results_Exp9_Tandem05_nodischadj_6_9.csv
               5 File(s)      4,359,242 bytes
```
As you can tell from the listing, this is on a Windows system. However, we want a solution that
works on Windows, Linux or Mac. The backslash vs forward slash issue in file paths can cause grief. When I [first developed a Python solution](http://hselab.org/concat-csv-pandas.html) for this problem (on a Linux system), I did the typical thing and used the built in `os` and `glob` libraries in Python to work with file paths and the file system. Since then, I've learned about the newish system library called `pathlib`. Let's start by checking out these libraries.


## Working with paths and folders and the os

For tasks like the one above and many similar such tasks, we need a way to work with the file system and file paths. We can use the `os` module for working with paths and doing things like navigating directories. In addition, the `glob` module is handy for getting lists of files based on a filename pattern.

An alternative to `os` and `glob` is the object oriented `pathlib` moddule. Let's take a quick look at both of these approaches. This is **NOT** meant to be a comprehensive tutorial on `pathlib`. There's a really good post on it in Practical Business Python entitled [Using Python's pathlib module](https://pbpython.com/pathlib-intro.html). And, a good two part series on `pathlib` was done by Python blogger, Trey Hunter:

* [Why you should be using pathlib](https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/)
* [No really, pathlib is great](https://treyhunner.com/2019/01/no-really-pathlib-is-great/)

Of course, the [official pathlib docs](https://docs.python.org/3/library/pathlib.html) are really useful - there's even a [table of os and pathlib equivalents](https://docs.python.org/3/library/pathlib.html#correspondence-to-tools-in-the-os-module).



### Using os and glob
This is just a glimpse at a few operations with `os`. One thing to note is that the result of things like `getcwd` is a string.

In [ ]:
import os
from glob import glob

In [ ]:
# Retrieve current working directory (`cwd`)
cwd = os.getcwd()
print(cwd)
print(type(cwd))

# List all files and directories in data directory
print(os.listdir("./data"))

# Use glob to get all the csv filenames
for csv_fn in glob("./data/*.csv"):
    print(csv_fn)


### Using pathlib

Now let's get a brief introduciton to `pathlib`. Again, check out the links above for thorough tutorials.

First, let's import the `Path` object from `pathlib`.

In [ ]:
from pathlib import Path

Get the current working directory. Notice that the result is not a string; it's a `WindowsPath` object. If we were on Linux, it would be a `PosixPath` object.

In [ ]:
Path.cwd()

You can build up path objects using the `/` operator.

In [ ]:
data_dir = Path.cwd() / "data"
data_dir

We can also build up paths using the `Path` object. Note in this example, we get a relative path.

In [ ]:
in_file_1 = Path("data", "data-text.csv")
in_file_1

If we wanted the absolute path associated with this relative path, we just use the `absolute` method.

In [ ]:
in_file_1.absolute()

We can get the parts of a path with the `parts` method.

In [ ]:
in_file_1.parts

Another way to build up path objects is with the `joinpath` method.

In [ ]:
in_file_2 = Path.cwd().joinpath("data").joinpath("data-text.csv")
in_file_2

To do globbing we can use `pathlib`'s `glob` and `rglob` methods. It's important to note that these return generators - i.e. they don't actually compute all the values until asked. The 'r' in `rglob` stands for *recursive* and means it will dive down into subdirectories looking for filenames that match the specified pattern. The `glob` version just looks in the current directory.

In [ ]:
data_dir.rglob('*.csv')

In [ ]:
for f in data_dir.rglob('*.csv'):
    print(f)

Let's repeat using `glob` and just print the fiename and its component parts instead of the whole path.

In [ ]:
for f in data_dir.glob('*.csv'):
    print(f"filename = {f.name}, stem = {f.stem}, suffix = {f.suffix}")

The `iterdir` method returns `Path` objects for everything in the directory from which it is used. It's somewhat like `os.listdir`.

In [ ]:
for d in data_dir.iterdir():
    print(type(d), d)

The `pathlib` module includes methods for common filesystem operations. For example, let's create a new directory called `csvs` in our current working directory. Then let's find and copy all of the csv files that are in the `data` folder or any subfolders inside of `data`. Unfortunately, there isn't an actual `copy` method in `pathlib` that copies files (including metadata). For that part we need to use the built in `shutil` library. The `pathlib` modules does have a `rename` method that essentially allows you to move files and `write_bytes` and `write_text` which allows you to copy file contents.

In [ ]:
import shutil

In [ ]:
# Create a Path object corresponding to the new directory name
csvs_dir = Path.cwd() / 'csvs'
print(csvs_dir)
# Make the new directory
csvs_dir.mkdir(exist_ok=True)  # Since we are just practicing, we'll overwrite the directory if it already exists.
# The Path module has several useful is_* functions
print(csvs_dir.is_dir())

Now we can use `rglob` along with `shutil.copy` to actually copy the files.

In [ ]:
for f in data_dir.rglob('*.csv'):
    shutil.copy(f, csvs_dir)

## Back to Example 1 - Concatenating csv files

Now that we know how to work with `pathlib`, let's sketch out a strategy for combining all of the csv files in the `sim_output` directory into a single consolidated csv file, named `sim_output.csv`. Let's put the consolidated file in the `data` folder. As mentioned earlier, the consolidate file should be sorted by `scenario` and `rep`.

* Create `Path` objects for the source and destination of our files
* Create an empty dictionary that will store the intermediate `DataFrame` objects as they get created.
* Use `pathlib.glob` to loop over all the csv files in the source directory
    - Get the filename stem of the current csv file
    - read the csv file into a pandas DataFrame
    - store the DataFrame in our dictionary using the stem as the key
* Use the pandas `concat` function to concatenate the dataframes in the dictionary, creating the consolidated `DataFrame`
* Sort the consolidated `DataFrame` by scenario and rep
* Export the consolidated `DataFrame` to a csv file in the desired location

In [ ]:
# Inputs
csvs_path = Path.cwd() / 'data' / 'sim_output'
dest_path = Path.cwd() / 'data' / 'sim_output.csv'

sortkeys = ['scenario', 'rep']

# Create empty dict to hold the DataFrames created as we read each csv file
dfs = {}

# Loop over all the csv files 
for csv_f in csvs_path.glob('*.csv'):
    # Split the filename off from csv extension. We'll use the filename
    # (without the extension) as the key in the dfs dict.
    fstem = csv_f.stem

    # Read the next csv file into a pandas DataFrame and add it to
    # the dfs dict.
    df = pd.read_csv(csv_f)
    dfs[fstem] = df

# Use pandas concat method to combine the file specific DataFrames into
# one big DataFrame.
bigdf = pd.concat(dfs)

# Since we didn't try to control the order in which the files were read,
# we'll sort the final DataFrame in place by the specified sort keys.
bigdf.sort_values(sortkeys, inplace=True)

# Export the final DataFrame to a csv file. Suppress the pandas index.
bigdf.to_csv(dest_path, index=False)

## Example 2 - Consolidating data from multiple sheets in a Excel file using pandas

If you reviewed the `reading_excel_data.ipynb` notebook (and the associated `reading_excel_data.py` file) , you saw that we had some stream temperature data on six different sheets. In that notebook and Python script, we used the xlrd library as well as the openpyxl library to consolidate the data into a single csv file. 

![sites_1_6](images/sites_1_6.png)

While libraries like openpyxl are quite powerful and let you really dig into the details of reading and writing Excel files with Python, sometimes you can get by with a simpler approach. In this case, each sheet is structured the same:

* header in first row
* data follows

Pandas includes a quite powerful and flexible `read_excel` function that is more than up to the task. Let's check it out at https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html.

Notice that the default `sheet_name=0` which means only the first sheet is read. To read all the sheets, we need `sheets=None` We have a header in the first row and `header=0` is default, so we can take the default. The following should read all the sheets and return an `OrderedDict` whose keys are the sheet names. 

Also, we see that `read_excel`, while happy to accept a filename, is also happy to accept at `Path` object.

In [ ]:
sites_1_6_xl = Path('data', 'sites_1_6.xlsx')
sites_1_6_consolidated_xl = Path('data', 'sites_1_6_consolidated.xlsx')
sites_1_6_consolidated_csv = Path('data', 'sites_1_6_consolidated.csv')  # We'll try this in a bit
sites_1_6_xl

In [ ]:
sites_all = pd.read_excel(sites_1_6_xl, sheet_name=None)

In [ ]:
sites_all.keys()

In [ ]:
sites_all['Site1']

Now let's add a new column to each dataframe corresponding to the sheet name and then consolidate all the dataframes into a single dataframe.

In [ ]:
sheet_names = sites_all.keys()
for sheet_name in sheet_names:
    # Add new col
    sites_all[sheet_name]['site'] = sheet_name

# Ignore the index so that we don't get a multi-index based on the sheet name
consolidated_df = pd.concat(sites_all, ignore_index=True)
consolidated_df

Now we can write it back out to an Excel file. Let's not include the index. After running the next command, we'll go look at the resulting Excel file.

In [ ]:
consolidated_df.to_excel(sites_1_6_consolidated_xl, index=None)

For another approach we could use the openpyxl library. While not really needed with this nicely formatted Excel file, sometimes we need finer control over what we are reading in from Excel. This simple example shows how to access specific sheets and cells. I've written it as a function that accepts filenames. Hmmm, I wonder if openpyxl is now open to accepting `Path` objects. Let's try it.

In [ ]:
import openpyxl
import csv

In [ ]:
def consolidate_logger_xlsheets_openpyxl(fn_xlsx, fn_csv, verbose=True):
    """
    Combine xlsx sheets and output as csv using openpyxl

    Args:
        fn_xlsx: filename of Excel file to process
        fn_csv:  filename of output CSV file
        verbose: If true (default), writes status messages
    Returns:
        nothing, just writes CSV file

    """
    # Open the workbook
    wb_sites = openpyxl.load_workbook(fn_xlsx)

    # Accumulate the rows (lists) into another list. 
    # We'll end up with a list of lists.
    data = []
    for ws in wb_sites:
        if verbose:
            print("openpyxl:", ws.title)

        site = ws.title

        # Get the rows for the current sheet
        #for row in ws.rows[1:]:
        for row in ws.iter_rows():
            ws_data = [cell.value for cell in row]
            ws_data.append(site)
            data.append(ws_data)

    if verbose:
        print("openpyxl:", "Total number of rows: ", len(data), "\n")


    # Write CSV file
    with open(fn_csv, "w") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [ ]:
consolidate_logger_xlsheets_openpyxl(sites_1_6_xl, sites_1_6_consolidated_csv, verbose=True)

Brilliant, it works! It seems like `pathlib` is getting good adoption and just this little bit I've used it has won me over.

## Dealing with wide data and a multi-row header - CHALLENGE
For the next Excel data wrangling task, you will do the work. In the `data` folder you'll find an Excel file named `temp_data_wide.xlsx`. This was actually a real file we received from Antarctica during our stream temperature related research project. Usually we would get the data from each temperature logger in a separate Excel or csv file. But, many times, the scientists collecting the data would do a little Excel data prep on their own and combine the files into a single Excel file with separate columns for each data logger. The example file I've provided looks like this:

![temp_data_wide.xlsx](images/temp_data_wide.png)

Our eventual goal is to read this Excel file and create the following outputs. Each should end up in the `output` folder. You can find examples of my solution outputs in that folder.

### Excel file in long form

We want a new Excel file that has the data in "long" form. In other words, it should look like the following. Notice the new column names - your should have these same column names.

![temp_data_long](images/temp_data_long.png)

**HINT:** `melt`

### Excel file containing summary stats

We also want an Excel file containing summary stats by `loc_id`. It should look like this:

![summary_stats.png](images/summary_stats.png)

**HINT:** `describe`

### A plot of the data

Finally, we want a faceted plot. It should look like this:

![temp_data_plot_soln.png](images/temp_data_plot_soln.png)

**HINT:** Seaborn



Just try to get each piece working in the order presented above. You do **NOT** need tools like openpyxl. A combination of pandas, pathlib, matplotlib, and Seaborn is enough to do this. 

**MORE ADVANCED** Once you've got all the pieces working and generating the correct outputs, then it's time to turn this into a reusable function. Our function would take the following inputs:

* **wide_xlfile** - a path object or Excel filename for the wide formatted data
* **long_xlfile** (default=None) - a path object or Excel filename for the long formatted Excel file to create
* **stats_xlfile** (default=None) - a path object or Excel filename for the summary stats Excel file to create
* **plot** (default=None) - a path object or png filename for the plot to create
* **\*\*read_excel_kwargs** Any other keyword arguments that you want to pass to the pandas read_excel function. 

This last `**read_excel_kwargs` will give us more flexibility in that we can specify whichever rows we want to skip and/or the location of the header (or any other valid arguments to the `read_excel` function.

The **wide_xlfile** argument is required. The other named arguments are optional and default to None. If their value is None, then our function won't output that particular entity.

### SOLUTION - it's at the bottom of the notebook

## Appending new spreadsheet data in consolidated Excel workbook

For our last example, we'll look at appending new data to a simple Excel workbook based "database". This is a common scenario in which you've got an ever growing Excel workbook that gets new data appended in it at some, often fixed, frequency (such as daily or monthly). We'll use data from a telephone ACD system. I've simplified the problem for this example by doing the data cleaning necessary to create the "new data" spreadsheet from a more complicated text file (if you took my MIS 4460/5460 class you might recall the Muddy Data Tutorial). We'll automate that step some other day; for now let's just focus on appending new data to an existing workbook in which the format of the new data already matches the format of the consolidated workbook (the "database"). You'll find both the database workbook and the new data workbook in the `data/acd` folder. You'll also find a backup copy of the database workbook in case we destroy the original with some bad code. :)

To pull this off we'll use a combination of pandas and openpyxl. In this example, we aren't going to worry much about data formatting. Both [openpyxl](https://openpyxl.readthedocs.io/en/stable/formatting.html#conditional-formatting) and [XlsxWriter](https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html) are capable of doing all kinds of formatting including Conditional Formatting. One drawback of XlsxWriter is that it cannot read or edit existing Excel files - it's just for writing new ones. That's why I tend to use openpyxl as my general Python tool for working with Excel files.

At a high level, we need to be able to append new data to a specific, to be determined, spot in an existing Excel workbook.

https://stackoverflow.com/questions/47737220/append-dataframe-to-excel-with-pandas

https://stackoverflow.com/questions/38074678/append-existing-excel-sheet-with-new-dataframe-using-python-pandas/38075046#38075046

### The pandas `ExcelWriter` object

Pandas has a [specialized `ExcelWriter` object](https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html) that can be used with its `to_excel` function. This object gives us the ability to write data in *append mode* as well as doing some basic date and datetime formatting. According to the docs, it should be used as a context manager - this ensures that the destination file is properly opened and then closed after writing is done. The code below is from the pandas docs - note the `mode=a` for appending.

```
with ExcelWriter('path_to_file.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='Sheet3')
```

### The basic approach

We will:

* open the workbook with the new data using `openpyxl.load_workbook`
* open the database workbook using `ExcelWriter`
* find the last row in the database workbook
* write the new data in the next available row in the database workbook

In [ ]:
str(Path('data', 'acd', 'acd_interval_base.xlsx'))

In [ ]:
from openpyxl import load_workbook

new_data_xl = Path('data', 'acd', 'acd_interval_20210312.xlsx')
database_xl = str(Path('data', 'acd', 'acd_interval_base.xlsx'))

dest_sheet_name = 'interval_db'

# Read the new data into a pandas dataframe
new_data_df = pd.read_excel(new_data_xl, sheet_name=0)

# Create the ExcelWriter
with pd.ExcelWriter(database_xl, engine='openpyxl', mode='a') as writer:

    # Open database workbook
    writer.book = load_workbook(database_xl)

    # Get last row in database workbook - this is an actual Excel row number
    startrow = writer.book[dest_sheet_name].max_row
    print(f"Last row of data: {startrow}")
    
    # Since pandas to_excel uses a 0 based index for the rows, we want to start
    # writing at startrow
    
    # Without this next line, to_excel will create a new sheet and put the data there.
    # See https://stackoverflow.com/questions/38074678/append-existing-excel-sheet-with-new-dataframe-using-python-pandas/38075046#38075046
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    
    # Now that the writer object is "aware" of the sheets it can access, to_excel seems to work fine
    new_data_df.to_excel(writer, sheet_name=dest_sheet_name, startrow=startrow, index=False, header=False)
    print(f"New last row of data: {writer.book[dest_sheet_name].max_row}")

Of course, this is just the start of something useful. We have no checks to make sure we don't append the same data twice, for example. We shouldn't be using Excel as a database anyway, but people do it. :) The StackOverflow link in the code above has a more complete function for this task in the accepted answer - very nice.

## More on using Python for similar Excel tasks
Chris Moffit, in his Practical Business Python blog ([roadmap to the posts](https://pbpython.com/roadmap.html)), has many terrific posts that show how to use Python for common Excel tasks. If you use Excel and Python, his blog is an absolute **must read**. The posts include:

* [Combining Data From Multiple Excel Files](https://pbpython.com/excel-file-combine.html) - file globbing, concatenating dataframes,
* [Reading Poorly Structured Excel Files with Pandas](https://pbpython.com/pandas-excel-range.html) - advanced use of `read_excel`, accessing ranges and Tables
* [Common Excel Tasks Demonstrated in Pandas](https://pbpython.com/excel-pandas-comp.html) - totals rows, fuzzy string matching
* [Common Excel Tasks Demonstrated in Pandas - Part 2](https://pbpython.com/excel-pandas-comp-2.html) - selection and filtering
* [Improving pandas Excel output](https://pbpython.com/improve-pandas-excel-output.html) - using XlsxWriter to format Excel workbooks from Python
* [Creating Advanced Excel Workbooks](https://pbpython.com/advanced-excel-workbooks.html) - XlsxWriter, inserting VBA from Python(!), using COM to merge sheets
* [Interactive Data Analysis with Python and Excel ](https://pbpython.com/xlwings-pandas-excel.html) - using xlwings to "glue" Python and Excel together, using sqlalchemy to interact with databases



## Dealing with wide data and a multi-row header - SOLUTION

First I got it working in steps. Then I turned it into a function.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Create Path objecst for Excel file to process, summary stats and plot to output
temp_data_wide_xl = Path('data', 'temp_data_wide.xlsx')
temp_data_long_xl = Path('output', 'temp_data_long_soln.xlsx')
temp_data_stats_xl = Path('output', 'temp_data_stats_soln.xlsx')
temp_data_plot_png = Path('output', 'temp_data_plot_soln.png')

In [ ]:
# Read into pandas with read_excel. Need to skip rows 1 and 2.
temp_data_wide_df = pd.read_excel(temp_data_wide_xl, header=0, skiprows=[1,2])
temp_data_wide_df

In [ ]:
# Rename the Label col to datetime
temp_data_wide_df.rename({'Label': 'datetime'}, axis=1, inplace=True)
temp_data_wide_df

In [ ]:
# Reshape the data from wide to long
temp_data_long_df = temp_data_wide_df.melt(id_vars=['datetime'], var_name='loc_id', value_name='temp_c')
temp_data_long_df.to_excel(temp_data_long_xl, index=None)
temp_data_long_df

In [ ]:
# Compute the summary stats
summary_stats = temp_data_long_df.groupby(['loc_id'])['temp_c'].describe()
summary_stats.to_excel(temp_data_stats_xl)
summary_stats

Finally create the faceted plot.

In [ ]:
temp_g = sns.FacetGrid(temp_data_long_df, row="loc_id", sharey=True, height=2.5, aspect=11.7/2.5)
temp_g = temp_g.map(plt.plot, "datetime", "temp_c")
temp_g.savefig(temp_data_plot_png)



* **wide_xlfile** - a path object or Excel filename for the wide formatted data
* **long_xlfile** (default=None) - a path object or Excel filename for the long formatted Excel file to create
* **stats_xlfile** (default=None) - a path object or Excel filename for the summary stats Excel file to create
* **plot** (default=None) - a path object or png filename for the plot to create

In [ ]:
def process_wide_temp_file(wide_xlfile, long_xlfile=None, stats_xlfile=None, plot=None, **read_excel_kwargs):
    """Process wide formatted stream temperature data file

    Parameters
    ----------
    wide_xlfile : Path object
        The wide formatted Excel data file
    long_xlfile : optional (default is None), Path object
        Long formatted Excel data file created if not None
    stats_xlfile : optional (default is None), Path object
        Excel file for summary stats created if not None
    plot : optional (default is None), Path object
        Time series temperature plot created if not None
    **read_excel_kwargs : keyword and values (can be dict)
        arguments which will be passed to `DataFrame.to_excel()`
                            

    Returns
    -------
    No return value
        Generates output files as detailed above
    """
    
    # Read into pandas with read_excel. 
    temp_data_wide_df = pd.read_excel(wide_xlfile, **read_excel_kwargs)
    
    # Rename the Label col to datetime
    temp_data_wide_df.rename({'Label': 'datetime'}, axis=1, inplace=True)
    
    # Reshape the data from wide to long
    temp_data_long_df = temp_data_wide_df.melt(id_vars=['datetime'], var_name='loc_id', value_name='temp_c')
    
    if long_xlfile is not None:
        temp_data_long_df.to_excel(long_xlfile, index=None)
    
    # Compute the summary stats
    if stats_xlfile is not None:
        summary_stats = temp_data_long_df.groupby(['loc_id'])['temp_c'].describe()
        summary_stats.to_excel(stats_xlfile)
        
    # Make the plot
    if plot is not None:
        temp_g = sns.FacetGrid(temp_data_long_df, row="loc_id", sharey=True, height=2.5, aspect=11.7/2.5)
        temp_g = temp_g.map(plt.plot, "datetime", "temp_c")
        temp_g.savefig(plot)

    
    

In [ ]:
temp_data_wide_xl = Path('data', 'temp_data_wide.xlsx')
temp_data_long_xl = Path('output', 'temp_data_long_test.xlsx')
temp_data_stats_xl = Path('output', 'temp_data_stats_test.xlsx')
temp_data_plot_png = Path('output', 'temp_data_plot_test.png')

In [ ]:
process_wide_temp_file(temp_data_wide_xl, long_xlfile=temp_data_long_xl, stats_xlfile=temp_data_stats_xl, plot=temp_data_plot_png, header=0, skiprows=[1,2])